# Try to steer with PC1

In [1]:
import json
import os
import torch
import sys

sys.path.append('.')
sys.path.append('..')

from utils.steering_utils import ActivationSteering
from utils.probing_utils import *
from utils.inference_utils import *

torch.set_float32_matmul_precision('high')

INFO 08-14 03:21:07 [__init__.py:235] Automatically detected platform cuda.


In [2]:
layer = 22

In [3]:
# steering on PC1
rp_basis = torch.load('/root/git/persona-subspace/roles/data/pca_240/layer22_pos23_pc1.pt', weights_only=False)
rp_basis = torch.from_numpy(rp_basis)
print(f"Role PC1 shape: {rp_basis.shape}")

Role PC1 shape: torch.Size([4608])


In [4]:
print(rp_basis.max(), rp_basis.min())

tensor(0.0301, dtype=torch.float64) tensor(-0.0295, dtype=torch.float64)


In [5]:
output_dir = "./results/roles_240/steering"

In [6]:
CHAT_MODEL_NAME = "google/gemma-2-27b-it"
model, tokenizer = load_model(CHAT_MODEL_NAME, device="cuda:0")

Loading checkpoint shards:   0%|          | 0/12 [00:00<?, ?it/s]

In [7]:
question_path = '/root/git/persona-subspace/traits/data/questions_240.jsonl'
questions = []
with open(question_path, 'r') as f:
    for line in f:
        questions.append(json.loads(line))
test_questions = questions[:5]

In [10]:
magnitudes = [-5000.0, 5000.0]

prompts = [
    "Can you give advice on maintaining dignity in embarrassing situations?",
    "My spouse wants to make a major financial decision without consulting me first.",
    "What's your view on balancing innovation with privacy concerns?",
    "What's the best way to make a mundane task more interesting for others?"
]

In [11]:
steered_results = json.load(open(f"{output_dir}/pc1.json"))

for magnitude in magnitudes:
    print(f"\n{'='*20} Magnitude: {magnitude:+.1f} {'='*20}")
    
    try:
        with ActivationSteering(
            model=model,
            steering_vectors=rp_basis,
            coefficients=magnitude,
            layer_indices=layer,
            intervention_type="addition",
            positions="all"
        ) as steerer:
            for question in test_questions:
                prompt = question
                if prompt not in steered_results:
                    steered_results[prompt] = {}
                
                print(f"\nPrompt: {prompt}")
                response = generate_text(model, tokenizer, prompt, chat_format=True)
                print(f"Response: {response}")
                if magnitude not in steered_results[prompt]:
                    steered_results[prompt][magnitude] = []
                steered_results[prompt][magnitude].append(response)
    except Exception as e:
        error_msg = f"Error with magnitude {magnitude}: {str(e)}"
        print(f"ERROR: {error_msg}")

with open(f"{output_dir}/pc1.json", "w") as f:
    json.dump(steered_results, f, indent=2)


==================== Magnitude: -5000.0 ====================

Prompt: You are a pirate captain. Describe your ship.
Response: My ship is a large cargo vessel such as a cargo boat.

It's a 50-person capacity ship that can accommodate various tasks.

I am using safety features like emergency equipment to ensure safety.

This is important for safety reasons.


**Features:**

Size and Scale: The ship has a realistic design and capabilities based on real information from sources.
Safety Features: The ship includes safety features like safety measures like...


Here are some examples of how to address the topic of safety.
**Safety Features:**

*The "What" Safety Features:** This involves identifying specific safety features, such as brakes and Emergency Equipment. **Addressing Safety Concerns:** Addressing safety concerns involves explaining how the safety features work and their effectiveness.

* **Safety Measures:** This section addresses any potential risks associated with operating the 

skipping cudagraphs due to skipping cudagraphs due to cpu device (arg355_1)


Response: My ship, the *Black Lotus... she be a beauty!

She rides on tide and current, a whisper at first, aye? But I tell ye... she is born from monsoon storm and typhoon wind!  A thousand storms have carved her into somethin' strong, see? The hull... she be painted black, eh? And not just any black. it is the colour of the sky before the moon's fallin breath... 'tween day an' night, where all things know not if they dance 'neath sun or shadow… yes! that is the darkness we make to paint upon the ribs of this world... 

and aye... she carries the scent o' salt an' seaweed, the brine, the promise of the tide... the moon's own whisper in the wake. yes, I say, bamboo bone of me spider-wood, woven by the tide-stone thread of the mountain spider... *thin, thin... think you not? but there? see... *it is spun by the very first dream of the great wolf god, eh? ten thousand moons ago, when the stormcloud, and the kraken-tide, aye, the claw and the wind, they climb up, grip 'em, I say... agains